In [2]:
# | export
import numpy as np
import pandas as pd

In [70]:
#| default_exp sequence_digestion_matrix

In [64]:
#| hide
import nbdev; nbdev.nbdev_export()

# Digestion profile per unique sequence
> functions to build the digestion profile per unique sequence

In [41]:
#| export
def count_sequences_from_mother(seqcell, mother_seq, n_sequences, mnase_array, population, aggregation_per_cell='mean'):
    """
    Return digestion mother sequence.
    columns: digestion from right to left
    rows: digestion from left to right
    """
    seqcell = seqcell[(seqcell['sort_population']==population)]
    mnase_array.sort()
    count_matrices_list = []

    row_name = []
    column_name = []
    
    for mnase in mnase_array:
        count_m = np.zeros((n_sequences, n_sequences))
        df = seqcell[(seqcell['units_mnase'] == mnase)]

        for i in range(0, count_m.shape[0]):
            seq = mother_seq[:-(i)] if i != 0 else mother_seq
            row_name.append(seq)
            for j in range(count_m.shape[1]):
                seqj = seq[j:]
                counts = df.loc[df['seq']==seqj, 'counts'].aggregate(aggregation_per_cell) #sum or average counts per cell
                count_m[i, j] = counts
                column_name.append(seqj)
        count_m[np.isnan(count_m)] = 0
        count_matrices_list.append(count_m)
    count_matrix_stacked = np.stack(count_matrices_list) # 
    return count_matrix_stacked, row_name, column_name

In [50]:
mnase_df = pd.DataFrame([1,2,3], columns=['units_mnase'])
mnase_df['counts'] = 1
sequences_df = pd.DataFrame(['ABCDE','ABC', 'BCD', 'CDE', 'CD'], columns=['seq'])
sequences_df['counts'] = 1
df = pd.merge(sequences_df, mnase_df, on='counts')
df['sort_population'] = 2

m, rows, columns = count_sequences_from_mother(df, 'ABCDE', 3, mnase_df['units_mnase'].values, 2, aggregation_per_cell='mean' )

# matrix:
#    | ABCDE BCDE CDE |
#    | ABCD  BCD  CD  | x mnases
#    | ABC   BC   C   |

assert (m[0,:,:] == np.array([[1, 0, 1], [0, 1, 1], [1, 0, 0]])).all()

In [66]:
#| export
def digestion_profile_from_count_matrix(count_matrix_stacked, mnases, digestion_direction, mother_seq, sequence_as_index=False):
    sum_counts = count_matrix_stacked.sum(axis=digestion_direction)

    if digestion_direction == 2: # digestion by end
        sequences_digested = [mother_seq] + [mother_seq[:-i] for i in range(1,sum_counts.shape[1])]
        sequences_digested.sort(key=len, reverse=True)
    elif digestion_direction == 1: # digestion by start
        sequences_digested = [mother_seq[i:] for i in range(sum_counts.shape[1])]
        sequences_digested.sort(key=len, reverse=True)
    else:
        raise ValueError('Index out of range, choose 1 for 5-3 digestion (start) or 2 for 3-5 digestion (end)')
    mnase_seq_df = pd.DataFrame(sum_counts, columns=sequences_digested, index=mnases)
    if sequence_as_index:
        mnase_seq_df = mnase_seq_df.transpose()
    return mnase_seq_df

In [68]:
dp

,ABCDE,BCDE,CDE
1,2.0,1.0,2.0
2,2.0,1.0,2.0
3,2.0,1.0,2.0


In [69]:
mother_seq = 'ABCDE'

dp = digestion_profile_from_count_matrix(m, mnase_df['units_mnase'].values, 1, mother_seq)
assert 1 == dp.loc[1, 'BCDE']

dp = digestion_profile_from_count_matrix(m, mnase_df['units_mnase'].values, 2, mother_seq)
assert 2 == dp.loc[1, 'ABCD']

In [65]:
dp

,1,2,3
ABCDE,2.0,2.0,2.0
ABCD,2.0,2.0,2.0
ABC,1.0,1.0,1.0
